In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.linear_model import Ridge, Lasso
from sklearn.model_selection import train_test_split

In [6]:
df = pd.read_csv('energydata_complete.csv')
df.head()

,date,Appliances,lights,T1,RH_1,T2,RH_2,T3,RH_3,T4,RH_4,T5,RH_5,T6,RH_6,T7,RH_7,T8,RH_8,T9,RH_9,T_out,Press_mm_hg,RH_out,Windspeed,Visibility,Tdewpoint,rv1,rv2
0,2016-01-11 17:00:00,60,30,19.89,47.596667,19.2,44.790000,19.79,44.730000,19.000000,45.566667,17.166667,55.20,7.026667,84.256667,17.200000,41.626667,18.2,48.900000,17.033333,45.53,6.600000,733.5,92.0,7.000000,63.000000,5.3,13.275433,13.275433
1,2016-01-11 17:10:00,60,30,19.89,46.693333,19.2,44.722500,19.79,44.790000,19.000000,45.992500,17.166667,55.20,6.833333,84.063333,17.200000,41.560000,18.2,48.863333,17.066667,45.56,6.483333,733.6,92.0,6.666667,59.166667,5.2,18.606195,18.606195
2,2016-01-11 17:20:00,50,30,19.89,46.300000,19.2,44.626667,19.79,44.933333,18.926667,45.890000,17.166667,55.09,6.560000,83.156667,17.200000,41.433333,18.2,48.730000,17.000000,45.50,6.366667,733.7,92.0,6.333333,55.333333,5.1,28.642668,28.642668
3,2016-01-11 17:30:00,50,40,19.89,46.066667,19.2,44.590000,19.79,45.000000,18.890000,45.723333,17.166667,55.09,6.433333,83.423333,17.133333,41.290000,18.1,48.590000,17.000000,45.40,6.250000,733.8,92.0,6.000000,51.500000,5.0,45.410389,45.410389
4,2016-01-11 17:40:00,60,40,19.89,46.333333,19.2,44.530000,19.79,45.000000,18.890000,45.530000,17.200000,55.09,6.366667,84.893333,17.200000,41.230000,18.1,48.590000,17.000000,45.40,6.133333,733.9,92.0,5.666667,47.666667,4.9,10.084097,10.084097


In [7]:
df.isna().sum().head()

date          0
Appliances    0
lights        0
T1            0
RH_1          0
dtype: int64

In [8]:
data = df.drop(['date','lights'],axis=1)
normalized_data = pd.DataFrame(MinMaxScaler().fit_transform(data),columns=data.columns)
x = normalized_data.drop('Appliances', axis=1)
y = normalized_data.Appliances
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.3,random_state=42)
model = LinearRegression().fit(x_train,y_train)
y_pred = model.predict(x_test)
pd.Series(y_pred).head()

0    0.033222
1    0.244116
2    0.034000
3    0.105377
4    0.023360
dtype: float64

In [9]:
mae = mean_absolute_error(y_test,y_pred)
round(mae,3)

0.05

In [10]:
rss = np.sum(np.square(y_test-y_pred))
round(rss,3)

45.348

In [11]:
mse = mean_squared_error(y_test,y_pred)
rmse = np.sqrt(mse)
round(rmse,3)

0.088

In [12]:
r2_score = r2_score(y_test,y_pred) #coefficient of determination 
round(r2_score,2)

0.15

In [13]:
x1 = np.array(normalized_data.T2).reshape(-1,1) 
y1 = normalized_data.T6
x1_train,x1_test,y1_train,y1_test = train_test_split(x1,y1,test_size=0.3, random_state=42)
model2 = LinearRegression().fit(x1_train,y1_train)
y1_pred = model2.predict(x1_test)
y1_pred

array([0.23928945, 0.46794238, 0.23108472, ..., 0.3001772 , 0.4297256 ,
       0.3217686 ])

In [14]:
from sklearn.metrics import r2_score

# coefficient of determination (variance or dist from actual y_test)
r_score = r2_score(y1_test,y1_pred)
round(r_score,2)

0.64

In [15]:
weight_df = pd.DataFrame(pd.Series(model.coef_,x_train.columns).reset_index
                         ())
weight_df.columns = ['Feature','weight']
weight_df

,Feature,weight
0,T1,-0.003281
1,RH_1,0.553547
2,T2,-0.236178
3,RH_2,-0.456698
4,T3,0.290627
5,RH_3,0.096048
6,T4,0.028981
7,RH_4,0.026386
8,T5,-0.015657
9,RH_5,0.016006


In [30]:
ridge_reg = Ridge(alpha=0.4).fit(x_train,y_train)
ridge_ypred = ridge_reg.predict(x_test)
pd.Series(ridge_ypred,name='y_pred')

0       0.033219
1       0.240438
2       0.034613
3       0.107071
4       0.023944
          ...   
5916    0.084344
5917    0.075207
5918    0.068724
5919    0.100255
5920    0.058512
Name: y_pred, Length: 5921, dtype: float64

In [26]:
ridge_mse = mean_squared_error(y_test,ridge_ypred)
ridge_rmse = np.sqrt(ridge_mse)
round(ridge_rmse,3)

0.088

In [31]:
lasso_model = Lasso(alpha=0.001).fit(x_train,y_train)
lasso_ypred = lasso_model.predict(x_test)
pd.Series(lasso_ypred,name='lasso_ypred')

0       0.073703
1       0.081435
2       0.077161
3       0.088481
4       0.070655
          ...   
5916    0.079210
5917    0.086004
5918    0.077928
5919    0.090344
5920    0.083593
Name: lasso_ypred, Length: 5921, dtype: float64

In [39]:
lasso_weight_df = pd.DataFrame(pd.Series(lasso_model.coef_,x_train.columns).reset_index())
lasso_weight_df.columns = ['Feature','weight']
lasso_weight_df

,Feature,weight
0,T1,0.000000
1,RH_1,0.017880
2,T2,0.000000
3,RH_2,-0.000000
4,T3,0.000000
5,RH_3,0.000000
6,T4,-0.000000
7,RH_4,0.000000
8,T5,-0.000000
9,RH_5,0.000000


In [32]:
lasso_rmse = np.sqrt(mean_squared_error(y_test,lasso_ypred))
round(lasso_rmse,3)

0.094